In [23]:
import cv2
from skimage.metrics  import structural_similarity
import os
import numpy as np
import time

In [24]:
#@title Определяем параметры
PATH = 'Video/'     #Путь к видеофайлам
JUMP = 100          #Для корректировки указателя позиции кадра
OFFSET = 0          #Стартуем нумерацию со смещением
SIM_TRSH = 0.991    #Допустимая степень похожести видео и картинок

In [25]:
#@title Сервисные функции

In [26]:
# Контекстный менеджер для измерения времени операций
class timex:
    def __enter__(self):
        # Фиксация времени старта процесса
        self.t = time.time()
        return self

    def __exit__(self, type, value, traceback):
        # Вывод времени работы
        print('Время обработки: {:.2f} с'.format(time.time() - self.t))

In [27]:
# Показать несколько картинок
import matplotlib.pyplot as plt
import math
%matplotlib inline
def show_images(cols, *images):
    num_images = len(images)
    fig, axes = plt.subplots(cols, math.ceil(num_images/cols), figsize=(10, 5))
    axs = np.ravel(axes)
    for i, image in enumerate(images):
        axs[i].imshow(image, cmap=plt.cm.gray)
        axs[i].axis('off')

    plt.tight_layout()
    plt.show()
    plt.close()

In [28]:
# Сравнить кадры степень похожести -Similarity если 1 кадры совпадают 0 нет совпадений
def frame_count_compare(frame1, frame2):
    frame1_g = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    frame2_g = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    (score, frame_diff) = structural_similarity(frame1_g,
        frame2_g, full=True)
#    if score != 1:
#        show_images(2,frame1,frame2,frame_diff)
    return score, frame_diff


In [29]:
# Иммитация разбора кадров программой Антона
def frame_read(pos,video):
    video.set(cv2.CAP_PROP_POS_FRAMES,(pos//JUMP*JUMP))
    if pos%JUMP:
        for i in range(int(pos%JUMP)):
            ok,frame = video.read()
    else:      
        ok, frame = video.read()
    return ok, frame

In [30]:
def check_video_jump(vid_name,jump):

    cap1 = cv2.VideoCapture(vid_name)
    frame_count = int(cap1.get(cv2.CAP_PROP_FRAME_COUNT))
    print('Frame count:', frame_count)
    cap1.set(cv2.CAP_PROP_POS_FRAMES,OFFSET)
   # ok,frame1 = cap1.read()
    for frame_pos in range (frame_count-OFFSET):
      ok, frame1 = frame_read(frame_pos,cap1)
      if not ok:
         return 0
    #while ok:
    #  for i in range(jump):
    #    ok,frame1 = cap1.read()
    #    if  ok == False:
    #       return 0

      frame_num = cap1.get(cv2.CAP_PROP_POS_FRAMES)
      cap1.set(cv2.CAP_PROP_POS_FRAMES,frame_num-1)
      ok2,frame2 =cap1.read()
      if not ok2:
         print('Read Fail')
         return 0
      similarity = frame_count_compare(frame1,frame2)
      if similarity < 1:
         print(f'Frame Number: {frame_num} Similarity:{similarity}')
         cap1.set(cv2.CAP_PROP_POS_FRAMES,frame_num)      #Если кадры не совпали, корректируем позицию

In [31]:
def check_video_image_similarity(name,show_diff = False):
    video = os.path.join(PATH, name+'.mp4')
    cap = cv2.VideoCapture(video)
    cap.set(cv2.CAP_PROP_POS_FRAMES,OFFSET)
    ok, frame = cap.read()
    if not ok:
        print(f'Can not open file: {video}')
    frame_num = 1
    diff = 0
    try:
        img = cv2.imread(os.path.join('Images',name,f'{name}_{frame_num:06d}.jpg'))
    except:
        print('image',os.path.join('Images',name,f'{name}_{frame_num:06d}.jpg'), 'not found')
    while ok:
        if not ok: break
        try:
            img = cv2.imread(os.path.join('Images',name,f'{name}_{frame_num:06d}.jpg'))
        except:
            print('image',os.path.join('Images',name,f'{name}_{frame_num:06d}.jpg'), 'not found')
            break
        similarity,frame_diff = frame_count_compare(frame,img)
        if similarity < SIM_TRSH:
            print(f'Frame Number: {frame_num} Similarity:{similarity}')
            show_images(1,frame,img, frame_diff)
            diff +=1
 #           cap.set(cv2.CAP_PROP_POS_FRAMES,frame_num-2)
        ok,frame =cap.read()
        frame_num += 1
    print(f'Найдено {diff} отличий')

In [32]:
'''
for file in os.listdir(PATH):
    filepath = PATH+file
    print(f'File:{file}')
    with timex():
     check_video_jump(filepath,JUMP)
'''

"\nfor file in os.listdir(PATH):\n    filepath = PATH+file\n    print(f'File:{file}')\n    with timex():\n     check_video_jump(filepath,JUMP)\n"

In [33]:
for file in os.listdir(PATH):
    print(f'File:{file}')
    with timex():
        name,_ = os.path.splitext(file)
        check_video_image_similarity(name)

File:Batch_Night_20_11.mp4
Найдено 0 отличий
Время обработки: 175.04 с
File:Batch_BO_13-05.mp4
Найдено 0 отличий
Время обработки: 1705.16 с
File:Batch_Night_20-11.mp4
Найдено 0 отличий
Время обработки: 1477.20 с
File:Batch_MOC_19_40.mp4
Найдено 0 отличий
Время обработки: 1685.14 с
File:Batch_CP_16_10.mp4
Найдено 0 отличий
Время обработки: 215.52 с
File:Batch_BO_13_05.mp4
Найдено 0 отличий
Время обработки: 242.69 с
File:Batch_CP_16-10.mp4
Найдено 0 отличий
Время обработки: 1817.57 с
File:Batch_MOC_19-40.mp4
Найдено 0 отличий
Время обработки: 200.82 с
